# Prediction demonstration
The notebook shows how to use the segmentation and hidden part reconstruction to predict bubbles in 2D images. 

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
from utils_Segmentation import combinedPrediction,load_img,get_UNet_tf,predict_UNet
from csbdeep.utils import normalize
from stardist.models import StarDist2D
import matplotlib.pyplot as plt
import pathlib
import tensorflow as tf
from utils_StarBub import HiddenReco,SaveCSV_List
from tqdm import tqdm
from stardist import random_label_cmap
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

lbl_cmap=random_label_cmap()

## Processing Settings and loading the Models
The models are expected to be in the Models directory situated in the same directory as the notebook. Otherwise the Model_dir needs to be changed. Using GPU is recommend for computational speed.

In [ ]:
base_dir=os.path.abspath('')
Model_dir='Models/'
use_gpu=True # Switch off to use CPU (slower prediction)

if use_gpu:
    physical_devices=tf.config.list_physical_devices('GPU')
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device,True)
else: 
    os.environ['CUDA_VISIBLE_DEVICES']='-1'
modelUNet,resizer,axes = get_UNet_tf(Model_dir+'Try_1695892977.135699/')
modelSD =StarDist2D(None, name='stardist', basedir=Model_dir+'SDmodel')
model = tf.keras.models.load_model(Model_dir+'RDCModel')


### Example: Predict on a single image
The example shows how to predict bubbles by first predicting the segmentations mask with the 'createLabelUNet' and 'combinedPrediction' functions. Afterwards the hidden bubble parts are reconstructed either with the RDC method or by ellipse fitting.

In [ ]:
Metric=6.7E-5 # Size of an image pixel in m 
boolplot=True # Use this for visual inspection.
useRDC=True # Switch on to use the RDC method, otherwise ellipse fitting will be used.

ImgDir=base_dir+'/Examples/00000.jpg'

x=load_img(ImgDir)
X=normalize(x if x.ndim==2 else x[...,0], 1,99.8, axis=(0,1))
imgMask=predict_UNet(X,modelUNet,resizer,axes)
labels,labelsSD=combinedPrediction(X,modelSD,imgMask)
if boolplot:
    if useRDC==False:
        %config InlineBackend.print_figure_kwargs={'bbox_inches': None}
    else:
        %config InlineBackend.print_figure_kwargs={'bbox_inches': 'tight'}        
    _, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(25,25))
    ax1.imshow(X,cmap='gray');ax1.imshow(imgMask,cmap='Reds',alpha=0.5)
    ax1.set_axis_off()
    ax1.set_title('UNet mask')
    ax2.imshow(X,cmap='gray');ax2.imshow(labels,cmap=lbl_cmap,alpha=0.5)
    ax2.set_axis_off()
    ax2.set_title('Segmentation mask')
    ax3.imshow(X,cmap='gray')
    ax3.set_axis_off() 
    ax3.set_title('Hidden part reconstruction')   
Bubbles=HiddenReco(labels,Metric,useRDC=useRDC,model=model,boolPlot=boolplot,ax=ax3)


### Example: Evaluating a set of images
This example shows how to evalute a set of images and saving the results.

In [ ]:
Metric=6.7E-5 # Size of an image pixel in m
useRDC=True # Switch on to use the RDC method, otherwise ellipse fitting will be used.
writeOutSingle=False # Toggle this parameter to save the results for every single image.

ImgDir=base_dir+'/Examples/'
Imgs=sorted(pathlib.Path(ImgDir).glob('*.jpg')) # In practice, use another file identifier here if you do not have jpg images

OutDir=ImgDir+'Results/'
if os.path.isdir(OutDir)==False:
    os.mkdir(OutDir) 

AllBubbles=[] 
for i,img in enumerate(tqdm(Imgs,desc='Progress')):
    x=load_img(img)
    X=normalize(x if x.ndim==2 else x[...,0], 1,99.8, axis=(0,1))
    imgMask=predict_UNet(X,modelUNet,resizer,axes)
    labels,_=combinedPrediction(X,modelSD,imgMask)
    Bubbles=HiddenReco(labels,Metric,timestep=i,useRDC=useRDC,model=model)
    if writeOutSingle:
        SaveCSV_List(Bubbles,OutDir,img.name,header='X [px],Y [px],Diameter [m],Major [m],Minor [m], Timestep [-]')
    AllBubbles+=Bubbles
SaveCSV_List(AllBubbles,OutDir,'AllBubbles',header='X [px],Y [px],Diameter [m],Major [m],Minor [m], Timestep [-]')

### Prepare for tracking
The following example shows how to prepare the *Nodes.pkl* file of the predictions that is needed for the tracking algorithm from *Fate of bubble clusters rising in quiescent liquid* [[Paper]](https://arxiv.org/abs/2306.02101)[[Code]](https://rodare.hzdr.de/record/2316)

In [ ]:
import pandas as pd
Metric=6.7E-5
ImgDir=base_dir+'/Examples/'
Imgs=sorted(pathlib.Path(ImgDir).glob('*.jpg'))
point_list=[]
for i,img in enumerate(tqdm(Imgs,desc='Progress')):
    x=load_img(img)
    X=normalize(x if x.ndim==2 else x[...,0], 1,99.8, axis=(0,1))
    imgMask=predict_UNet(X,modelUNet,resizer,axes)
    labels,_=combinedPrediction(X,modelSD,imgMask)
    Bubbles=HiddenReco(labels,Metric,useRDC=True,model=model,tracking=True)
    df=pd.DataFrame({'frame':i,'detection_id':[b.ID for b in Bubbles],'posx':[b.Position[1] for b in Bubbles],'posy':[b.Position[0] for b in Bubbles],
    'feat':[b.points for b in Bubbles],'size':[b.Diameter for b in Bubbles],'bb_left':[b.Bbox[0] for b in Bubbles]
    ,'bb_top':[b.Bbox[1] for b in Bubbles],'bb_right':[b.Bbox[2] for b in Bubbles],'bb_bot':[b.Bbox[3] for b in Bubbles],'tN':[b.occu_rate for b in Bubbles],
    'Dia':[b.Dia for b in Bubbles],'Grad':[b.grad for b in Bubbles]})
    point_list.append(df)
nodes=pd.concat(point_list)
nodes.to_pickle(ImgDir+"/Nodes.pkl")

*Note* The images in the Examples folder have a too large pixel distance for the tracker of *Fate of bubble clusters rising in quiescent liquid*. Please refer to example provided in the [[code repository]](https://rodare.hzdr.de/record/2316) for a suitable bubble displacement. 